<a href="https://colab.research.google.com/github/DongUk-Park/RnD/blob/main/POI/NAIS/NAIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [ ]:
import csv
import numpy as np
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import time

## Data Utils

In [ ]:
def load_data():
  train_dataset_path = '/content/history_train.csv'
  validation_dataset_path = '/content/history_validation.csv'
  test_dataset_path = '/content/history_test.csv'

  train_history_data = []
  validation_history_data = []
  test_history_data = []

  with open(train_dataset_path, 'r', newline='') as csv_train:
      csv_reader = csv.reader(csv_train)
      header = next(csv_reader)  # 헤더 행, 실제론 data 바로 들어옴
      header = [int(item) for item in header]
      train_history_data.append(header)
      for row in csv_reader:
          r = [int(item) for item in row]
          train_history_data.append(r)

  with open(validation_dataset_path, 'r', newline='') as csv_validation:
      csv_reader = csv.reader(csv_validation)
      header = next(csv_reader)  # 헤더 행, 실제론 data 바로 들어옴
      header = [int(item) for item in header]
      validation_history_data.append(header)
      for row in csv_reader:
          r = [int(item) for item in row]
          validation_history_data.append(r)

  with open(test_dataset_path, 'r', newline='') as csv_test:
      csv_reader = csv.reader(csv_test)
      header = next(csv_reader)  # 헤더 행, 실제론 data 바로 들어옴
      header = [int(item) for item in header]
      test_history_data.append(header)
      for row in csv_reader:
          r = [int(item) for item in row]
          test_history_data.append(r)
  business_id_info = pd.read_csv('/content/business_info_in_philadelphia.csv')
  item_list = business_id_info.index.tolist()


  # print("여기는 함수한")
  # print(business_id_info)
  # print(item_list)
  # print(train_history_data[0])

  num_users = len(train_history_data) # 필라델피아의 사용자 수 : 15919명
  num_items = business_id_info.shape[0] # 필라델피아의 가게 수 : 14603개 , 마지막 business_id : 150336
  return item_list, num_users, num_items, train_history_data, validation_history_data, test_history_data

In [ ]:
def get_train_data_user(item_list, num_users, train_list, num_negatives):
    """
    학습데이터를 확인하고 num_negatives만큼의 negative데이터를 뽑아내 학습에 사용
    negative sample
    """
    num_items = len(item_list)
    user_input, item_input, labels, batch_length = [],[],[],[]

    for u in range(num_users):
       np.random.shuffle(train_list[u])
       if u == 0:
           batch_length.append((1+num_negatives) * len(train_list[u]))
       else:
           batch_length.append((1+num_negatives) * len(train_list[u])+batch_length[u-1])

       for i in train_list[u]:
            # positive instance
            user_input.append(u)
            item_input.append(i)
            labels.append(1)
            # negative instances
            for t in range(num_negatives):
                j_idx = np.random.randint(0, num_items) # num_items == len(item_list), item_list에서 Negative sample을 골라야하기 때문에 인덱스를 골라줌
                j = item_list[j_idx]
                while j in train_list[u]:
                    j_idx = np.random.randint(0, num_items)
                    j = item_list[j_idx]
                user_input.append(u)
                item_input.append(j)
                labels.append(0)
    return user_input, item_input, labels, batch_length

def get_input_data(batch_length, train_list, user_input, item_input, batch):
    """
    recall, precision을 위한 입력 데이터 생성
    """
    p=0
    target_idx = []
    if batch == 0:
        p = 0
    else:
        p = batch_length[batch-1] # p : 지난 배치를 지나감

    target_idx = item_input[p:batch_length[batch]]

    temp = set(train_list[user_input[p]])

    history = []
    for idx in range(batch_length[batch]-p):
        history.append(list(temp))
    p=batch_length[batch]

    return history, target_idx


## Evaluate

In [ ]:
def eval(model, num_users, testPositives, history_list, target_idx_list):
    model.eval()
    hit = [0,0,0,0,0]
    precision = [0,0,0,0,0]
    recall = [0,0,0,0,0]

    k=[10,20,30,40,50]
    a = 0.9

    with torch.no_grad():
        for u in range(num_users):
            count = 0
            history = history_list[u]
            target_idx = target_idx_list[u]
            results=model(history, target_idx)

            for i in range(len(k)):
                topk = torch.topk(results,k[i])
                topSet = set(target_idx[topk.indices.tolist()])

                positives = set(testPositives[u])
                count = len(topSet & positives)

                precision[i] += (count/k[i])
                recall[i] += (count/len(testPositives[u]))
                if count >0:
                    hit[i] += 1

        for i in range(len(k)):
            recall[i]/= num_users
            precision[i]/= num_users
            hit[i]/= num_users

    return recall , precision, hit


## Train

In [28]:
def train(model, epoch, history, user_input, item_input, labels, batch_length, optimizer, validation_history_data, user_input_validation, item_input_validation, labels_validation, batch_length_validation, test_history_data):
    loss_func = nn.BCELoss()
    before_loss = 210000000.0
    max_re = 0.0

    recall = []
    prec = []
    hit = []

    for e in range(epoch):
        model.train()
        train_loss = 0.0
        start_time = int(time.time())

        p = 0 # 지난 배치는 지나가도록 확인해주는 변수

        for b in range(len(batch_length)): # user 수, 즉 배치 개수 만큼 반복
          print(f"User : {b}")
          batch_size = batch_length[b]
          user = []
          for i in range(5): # 5 : negative_num + 1
            user.extend(history[b])

          #print(f"user_len : {len(user)}")
          #print(f"p: {p}, batch_size : {batch_size}")
          item = item_input[p:batch_size]

          print(f"len_item : {len(item)}")
          optimizer.zero_grad()
          prediction = model(user, item)
          temp = torch.tensor(labels[p:batch_size], dtype=torch.float32)

          #print(prediction)
          #print(temp)
          loss = model.loss_func(prediction,temp)

          p = batch_size # p 업데이트
          loss.backward()
          train_loss += loss.item()
          optimizer.step()

        end_time = int(time.time())

        print("Train Epoch: {}; time: {} sec; loss: {:.4f}".format(e+1, end_time-start_time,train_loss))

        re,pr,hi = eval(model, num_users, validation_history_data, train_history_data, item_input_validation)
        recall.append(re)
        prec.append(pr)
        hit.append(hi)

        print("recall: {} precision: {} hit: {};".format( re[0],pr[0],hi[0]))
        re,pr,hi,re_d,pr_d,hi_d = eval(model,num_users, test_history_data, train_history_data, item_input_test)

        recall.append(re)
        prec.append(pr)
        hit.append(hi)


        end_time = int(time.time())
        print("time: {} sec;".format( end_time-start_time))

    return recall , prec, hit


## NAIS Model Define

In [29]:
class NAIS_basic(nn.Module):
    def __init__(self, item_num, embed_size, beta): # embed_size : 64, beta : 0.5
        super(NAIS_basic, self).__init__()
        self.embed_size = embed_size # concat 연산 시 * 2
        self.item_num = item_num
        self.beta = beta

        self.embed_history = nn.Embedding(item_num, embed_size) # (m:14603 * d:64), 과거 방문한 데이터(q), 유저별로 각각 하나씩 가져야하나 ?
        self.embed_target = nn.Embedding(item_num, embed_size) # (m:14603 * d:64), 예측 데이터(p)


        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.loss_func = nn.BCELoss() # binary cross entropy

        # Attention을 위한 MLP Layer 생성
        self.attn_layer1 = nn.Linear(embed_size, embed_size)
        self.attn_layer2 = nn.Linear(embed_size, 1, bias = False)

        self._init_weight_()

    def _init_weight_(self):
        # weight 초기화, 표준편차 : 0.01
        nn.init.normal_(self.embed_history.weight, std=0.01)
        nn.init.normal_(self.embed_target.weight, std=0.01)

        # bias 초기화
        for m in self.modules():
            if isinstance(m, nn.Linear) and m.bias is not None:
                m.bias.data.zero_()

    def forward(self, history, target):
        #배치 사이즈만큼 잘라서 넣어줌
        #print(len(history), len(target))
        history_tensor = torch.LongTensor(history)
        target_tensor = torch.LongTensor(target)
        #print(f"history_tensor : {history_tensor.shape}")
        #print(f"target_tensor : {target_tensor.shape}")

        prediction = self.attention_network(history_tensor,target_tensor)
        return self.sigmoid(prediction)

    def attention_network(self, user_history, target_item):
        """
        b: batch size (= input_item_num)
        h: history size (h * 5 = item_num = batch_size)
        d: embedding size
        """
        #print("#### def attention network start #####")
        #print(len(user_history), user_history)
        #print(len(target_item), target_item)

        history = self.embed_history(user_history) # (b * d)
        target = self.embed_target(target_item) # (b * d)
        batch_dim = len(target) # (b)

        #print(f"history_embedding : {history.shape}")
        #print(f"target_embedding : {target.shape}")
        #print(f"batch_dim : {batch_dim}")

        target = torch.reshape(target,(batch_dim, 1,-1))
        #print(f"target_reshaped to batch_dim : {target.shape}") # (b * 1 * d)

        input = history * target # (b*d) * (b*1*d) => (b * b * d)
        #print(f"input_data : {input.shape}")

        #print("#### attention layer start")
        attention_result = self.relu(self.attn_layer1(input)) # (b * b * d)
        #print(f"first attention : {attention_result.shape}")

        attention_result = self.attn_layer2(attention_result) # (b * b * 1)
        #print(f"second attention : {attention_result.shape}")

        exp_A = torch.exp(attention_result) # (b * b * 1)
        #print(f"exp_A : {exp_A.shape}")
        exp_A = exp_A.squeeze(dim=-1)# (b * b)
        #print(f"exp_A_Squeeze : {exp_A.shape}")

        mask = self.get_mask(user_history,target_item) # (b * b)
        #print(f"mask : {mask.shape}")
        exp_A = exp_A * mask # (b * b)
        #print(f"exp_A_mask : {exp_A.shape}")
        exp_sum = torch.sum(exp_A,dim=-1) # (b)
        #print(f"exp_A_sum : {exp_sum.shape}")
        exp_sum = torch.pow(exp_sum, self.beta) # (b)
        #print(f"exp_A_pow : {exp_sum.shape}")

        attn_weights = torch.divide(exp_A.T,exp_sum).T # (b * b)
        #print(f"attn_weights = {attn_weights.shape}")
        attn_weights = attn_weights.reshape([batch_dim,-1, 1])# (b * b * 1)
        #print(f"attn_weights_reshaped = {attn_weights.shape}")
        result = history * attn_weights# (b * b * d)
        target = target.reshape([batch_dim,-1,1]) # (b * d * 1)


        prediction = torch.bmm(result, target).squeeze(dim=-1) # (b * b * 1) -> (b * b)
        prediction = torch.sum(prediction, dim = -1) # (b)
        #print(f"#### return prediction{prediction.shape}")
        return prediction

    def get_mask(self, user_history, target_item):
        target_item = target_item.reshape([len(target_item),1])
        mask = user_history != target_item
        return mask
    def loss_function(self, prediction, label):
        return self.loss_func(prediction, label)

## Main

In [ ]:
if __name__ == "__main__":
    # USE_CUDA = torch.cuda.is_available()
    # DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
    #print("사용하는 Device : ", DEVICE)
    epoch = 40
    num_negatives = 4

    item_list, num_users, num_items, train_history_data, validation_history_data, test_history_data = load_data()
    user_input_train, item_input_train, labels_train, batch_length_train = get_train_data_user(item_list, num_users, train_history_data, num_negatives)
    user_input_validation, item_input_validation, labels_validation, batch_length_validation = get_train_data_user(item_list, num_users, validation_history_data, num_negatives)
    user_input_test, item_input_test, labels_test, batchlength_test = get_train_data_user(item_list, num_users, test_history_data, num_negatives)
    print(num_items, num_users)
    print(len(item_list), item_list[0], item_list[-1])

    for i in range(5):
        nais_model_name = "NAIS_basic_product"

        nais_model = NAIS_basic(num_items, 64, 0.5) # num_items, embed_size, beta
        optimizer1 = optim.Adagrad(nais_model.parameters(), lr=0.005, weight_decay=0.001)
        print(nais_model_name + " model")

        recall, prec, hit, recall_d, prec_d, hit_d = train(nais_model, epoch, train_history_data, user_input_train, item_input_train, labels_train, batch_length_train, optimizer1, validation_history_data, user_input_validation, item_input_validation, labels_validation, batch_length_validation, test_history_data)

        with open("/content/result/"+nais_model_name+ str(i)+".txt", mode='wt') as f:
          for g in range(len(recall)):
              f.write("epoch: "+str((g+1))+"-----------------------------------------\n")
              f.write("recall:"+ str( recall[g])+", precision: "+ str(prec[g])+", hit: "+ str(hit[g])+"\n")


14585 15919
14585 0 14584
NAIS_basic_product model
User : 0
len_item : 50
User : 1
len_item : 50
User : 2
len_item : 40
User : 3
len_item : 35
User : 4
len_item : 30
User : 5
len_item : 30
User : 6
len_item : 35
User : 7
len_item : 30
User : 8
len_item : 45
User : 9
len_item : 60
User : 10
len_item : 230
User : 11
len_item : 35
User : 12
len_item : 30
User : 13
len_item : 45
User : 14
len_item : 30
User : 15
len_item : 160
User : 16
len_item : 30
User : 17
len_item : 90
User : 18
len_item : 35
User : 19
len_item : 50
User : 20
len_item : 210
User : 21
len_item : 35
User : 22
len_item : 30
User : 23
len_item : 55
User : 24
len_item : 35
User : 25
len_item : 35
User : 26
len_item : 45
User : 27
len_item : 30
User : 28
len_item : 50
User : 29
len_item : 135
User : 30
len_item : 45
User : 31
len_item : 165
User : 32
len_item : 60
User : 33
len_item : 35
User : 34
len_item : 90
User : 35
len_item : 40
User : 36
len_item : 35
User : 37
len_item : 30
User : 38
len_item : 185
User : 39
len_ite